In [1]:
import re

In [2]:
airlines_rdd = sc.textFile('data/ontime/airlines.csv')
airlines_rdd.cache()
airlines_rdd.count()

1579

In [3]:
airlines_rdd.map(lambda l: l.split(',')).take(10)
airlines = airlines_rdd.map(lambda l: re.sub(r'"', '', l)).map(lambda l: l.split(','))
airlines.take(10)

[[u'19031', u'Mackey International Inc.: MAC'],
 [u'19032', u'Munz Northern Airlines Inc.: XY'],
 [u'19033', u'Cochise Airlines Inc.: COC'],
 [u'19034', u'Golden Gate Airlines Inc.: GSA'],
 [u'19035', u'Aeromech Inc.: RZZ'],
 [u'19036', u'Golden West Airlines Co.: GLW'],
 [u'19037', u'Puerto Rico Intl Airlines: PRN'],
 [u'19038', u'Air America Inc.: STZ'],
 [u'19039', u'Swift Aire Lines Inc.: SWT'],
 [u'19040', u'American Central Airlines: TSF']]

In [4]:
flight_rdd = sc.textFile('data/ontime/flights.csv')
flight_rdd.cache()

MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:-2

In [5]:
flight_rdd.take(10)

[u'2014-04-01,19805,1,JFK,LAX,0854,-6.00,1217,2.00,355.00,2475.00',
 u'2014-04-01,19805,2,LAX,JFK,0944,14.00,1736,-29.00,269.00,2475.00',
 u'2014-04-01,19805,3,JFK,LAX,1224,-6.00,1614,39.00,371.00,2475.00',
 u'2014-04-01,19805,4,LAX,JFK,1240,25.00,2028,-27.00,264.00,2475.00',
 u'2014-04-01,19805,5,DFW,HNL,1300,-5.00,1650,15.00,510.00,3784.00',
 u'2014-04-01,19805,6,OGG,DFW,1901,126.00,0640,95.00,385.00,3711.00',
 u'2014-04-01,19805,7,DFW,OGG,1410,125.00,1743,138.00,497.00,3711.00',
 u'2014-04-01,19805,8,HNL,DFW,1659,4.00,0458,-22.00,398.00,3784.00',
 u'2014-04-01,19805,9,JFK,LAX,0648,-7.00,1029,19.00,365.00,2475.00',
 u'2014-04-01,19805,10,LAX,JFK,2156,21.00,0556,1.00,265.00,2475.00']

In [7]:
flight_rdd.count()

476881

In [37]:
from datetime import datetime
from operator import add
from __future__ import division

In [11]:
airline_over10 = sqlContext.sql("SELECT airline FROM flight WHERE dep_delay >= 10")
airline_over10.count()

110223

In [43]:
airline_under0 = sqlContext.sql("SELECT airline FROM flight WHERE dep_delay < 0")
airline_under0.count()

273610

In [18]:
flight_airline = flight_rdd.map(lambda l: l.split(',')).map(lambda a: (a[1], float(a[6]))).reduceByKey(add).join(airlines)

In [22]:
flight_airline.takeOrdered(10, lambda s: -s[1][0])

[(u'19393', (1321080.0, u'Southwest Airlines Co.: WN')),
 (u'20366', (681665.0, u'ExpressJet Airlines Inc.: EV')),
 (u'19790', (366658.0, u'Delta Air Lines Inc.: DL')),
 (u'19977', (346069.0, u'United Air Lines Inc.: UA')),
 (u'20304', (301694.0, u'SkyWest Airlines Inc.: OO')),
 (u'19805', (291244.0, u'American Airlines Inc.: AA')),
 (u'20398', (242241.0, u'Envoy Air: MQ')),
 (u'20409', (171483.0, u'JetBlue Airways: B6')),
 (u'20355', (136810.0, u'US Airways Inc.: US')),
 (u'20436', (51949.0, u'Frontier Airlines Inc.: F9'))]

In [34]:
flight_count = flight_rdd.map(lambda l: l.split(',')).map(lambda a: (a[1], 1)).reduceByKey(add)#.takeOrdered(10, lambda k: -k[1])

In [62]:
flight_delay = flight_rdd.map(lambda l: l.split(',')).map(lambda a: (a[1], float(a[6]))).reduceByKey(add)

In [63]:
flgight_avg_dalay = flight_delay.join(flight_count)
flgight_avg_dalay.take(10)

[(u'19805', (291244.0, 43256)),
 (u'20304', (301694.0, 49848)),
 (u'19393', (1321080.0, 98371)),
 (u'20366', (681665.0, 55437)),
 (u'20436', (51949.0, 5960)),
 (u'20355', (136810.0, 34266)),
 (u'19977', (346069.0, 39244)),
 (u'19690', (-12936.0, 5885)),
 (u'20409', (171483.0, 20633)),
 (u'20437', (41718.0, 8163))]

In [64]:
flgight_avg_dalay.map(lambda i: (i[0], i[1][0]/i[1][1])).join(airlines).takeOrdered(10, lambda k: -k[1][0])

[(u'19393', (13.429567657134724, u'Southwest Airlines Co.: WN')),
 (u'20366', (12.296210112379818, u'ExpressJet Airlines Inc.: EV')),
 (u'19977', (8.818392620527979, u'United Air Lines Inc.: UA')),
 (u'20436', (8.716275167785234, u'Frontier Airlines Inc.: F9')),
 (u'20409', (8.31110357194785, u'JetBlue Airways: B6')),
 (u'20398', (7.372135487994157, u'Envoy Air: MQ')),
 (u'21171', (6.989682212133719, u'Virgin America: VX')),
 (u'19805', (6.733031255779545, u'American Airlines Inc.: AA')),
 (u'20304', (6.05227892794094, u'SkyWest Airlines Inc.: OO')),
 (u'19790', (5.597661140117859, u'Delta Air Lines Inc.: DL'))]

In [7]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [50]:
flights = flight_rdd.map(lambda l: l.split(',')).map(lambda l: Row(d=datetime.strptime(l[0], "%Y-%m-%d"),
                                                                  airline=l[1],
                                                                  flightnum=l[2],
                                                                  origin=l[3],
                                                                  dest=l[4],
                                                                  dep=float(l[5]),
                                                                  dep_delay=float(l[6]),
                                                                  arv=float(l[7]),
                                                                  arv_delay=float(l[8]),
                                                                  airtime=float(l[9]),
                                                                  distance=float(l[10])))
flight_df = sqlContext.createDataFrame(flights)
# schemaFlight.registerTempTable("flight")



In [51]:
type(flight_df)

pyspark.sql.dataframe.DataFrame

In [52]:
flight_df.count()

476881

In [53]:
flight_df.corr('dep_delay', 'distance')

0.0011992737446144216

In [54]:
flight_df.corr('dep_delay', 'arv_delay')

0.9423668636533158

In [55]:
df = flight_df.toPandas()

In [56]:
type(df)

pandas.core.frame.DataFrame